In [1]:
import pandas as pd
import numpy as np


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/My Drive/plant-pathology-2020-fgvc7.zip"

Archive:  /content/drive/My Drive/plant-pathology-2020-fgvc7.zip
  inflating: images/Test_0.jpg       
  inflating: images/Test_1.jpg       
  inflating: images/Test_10.jpg      
  inflating: images/Test_100.jpg     
  inflating: images/Test_1000.jpg    
  inflating: images/Test_1001.jpg    
  inflating: images/Test_1002.jpg    
  inflating: images/Test_1003.jpg    
  inflating: images/Test_1004.jpg    
  inflating: images/Test_1005.jpg    
  inflating: images/Test_1006.jpg    
  inflating: images/Test_1007.jpg    
  inflating: images/Test_1008.jpg    
  inflating: images/Test_1009.jpg    
  inflating: images/Test_101.jpg     
  inflating: images/Test_1010.jpg    
  inflating: images/Test_1011.jpg    
  inflating: images/Test_1012.jpg    
  inflating: images/Test_1013.jpg    
  inflating: images/Test_1014.jpg    
  inflating: images/Test_1015.jpg    
  inflating: images/Test_1016.jpg    
  inflating: images/Test_1017.jpg    
  inflating: images/Test_1018.jpg    
  inflating: images/Tes

In [6]:
test_df = pd.read_csv('test.csv')

In [5]:
submission_df=pd.read_csv('test.csv')
maxtrix=np.array(submission_df)
first_col= maxtrix[:,0]
first_col

array(['Test_0', 'Test_1', 'Test_2', ..., 'Test_1818', 'Test_1819',
       'Test_1820'], dtype=object)

In [7]:
train_df= pd.read_csv("train.csv")
images_id_csv = train_df['image_id']
healthy_csv = train_df['healthy']
multiple_diseases_csv = train_df['multiple_diseases']
rust_csv = train_df['rust']
scab_csv = train_df['scab']




In [8]:
print(images_id_csv[0],
healthy_csv[0],
multiple_diseases_csv[0],
rust_csv[0],
scab_csv[0])

Train_0 0 0 0 1


In [9]:
images_path = glob.glob('images' + '/*.jpg')
images_path

['images/Train_1040.jpg',
 'images/Train_608.jpg',
 'images/Test_730.jpg',
 'images/Train_406.jpg',
 'images/Train_128.jpg',
 'images/Test_846.jpg',
 'images/Train_717.jpg',
 'images/Test_213.jpg',
 'images/Train_1599.jpg',
 'images/Train_1090.jpg',
 'images/Train_1386.jpg',
 'images/Train_400.jpg',
 'images/Test_1519.jpg',
 'images/Train_1327.jpg',
 'images/Train_634.jpg',
 'images/Train_1751.jpg',
 'images/Train_326.jpg',
 'images/Train_1615.jpg',
 'images/Train_1475.jpg',
 'images/Train_550.jpg',
 'images/Test_122.jpg',
 'images/Train_255.jpg',
 'images/Train_619.jpg',
 'images/Test_638.jpg',
 'images/Test_225.jpg',
 'images/Train_681.jpg',
 'images/Test_1318.jpg',
 'images/Test_90.jpg',
 'images/Test_1412.jpg',
 'images/Test_1016.jpg',
 'images/Train_96.jpg',
 'images/Test_223.jpg',
 'images/Test_1746.jpg',
 'images/Train_339.jpg',
 'images/Train_1657.jpg',
 'images/Train_484.jpg',
 'images/Train_1155.jpg',
 'images/Test_9.jpg',
 'images/Test_1070.jpg',
 'images/Test_993.jpg',
 'im

In [10]:
train_labels = train_df.loc[:,['healthy','multiple_diseases','rust','scab']]
train_labels.to_numpy()

array([[0, 0, 0, 1],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       ...,
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

In [11]:

train_data = []
for i in range(len(train_df)):
    img_path = 'images/' + train_df['image_id'][i] + '.jpg'
    img = cv2.imread(img_path)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_resized = cv2.resize(img_gray, (128, 128))
    hog = cv2.HOGDescriptor()
    hog_feature = hog.compute(img_resized)
    train_data.append(hog_feature.flatten())
    
X_train = np.array(train_data)
y_train = np.array(train_labels)

test_data = []
test_labels = []
for i in range(len(test_df)):
    img_path = 'images/' + test_df['image_id'][i] + '.jpg'
    img = cv2.imread(img_path)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_resized = cv2.resize(img_gray, (128, 128))
    hog = cv2.HOGDescriptor()
    hog_feature = hog.compute(img_resized)
    test_data.append(hog_feature.flatten())
    
X_test = np.array(test_data)

In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
# y_train = np.argmax(y_train, axis=1)
y_train = np.array(train_labels)
y_train

array([[0, 0, 0, 1],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       ...,
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1]])

In [14]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier

svm =  OneVsRestClassifier(SVC(kernel='linear', C=1, probability=True))
svm.fit(X_train, y_train)

OneVsRestClassifier(estimator=SVC(C=1, kernel='linear', probability=True))

In [15]:
from sklearn.metrics import accuracy_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score
y_pred = svm.predict_proba(X_test)



In [16]:
y__= np.argmax(y_pred, axis=1)
y_train__=np.argmax(y_train, axis=1)


In [17]:
mask = y_train__ == y__

In [18]:
accuracy = mask.sum() / y__.shape[0]
print('Accuracy:', accuracy)

Accuracy: 0.31960461285008235


In [29]:
first_col=np.array(first_col)
y_pred=np.array(y_pred)


In [34]:

value_df = pd.DataFrame(y_pred,columns=['healthy', 'multiple_diseases', 'rust','scab'])

col_df = pd.DataFrame(first_col,columns=['image_id'])

In [36]:
result_df=pd.concat([col_df, value_df], axis=1)

result_df.to_csv('submission.csv', index= False )
